# BM25 Retrieval with PyTerrier

### Step 1: Import everything and load variables

In [8]:
import pyterrier as pt
import pandas as pd

# We use three methods from the tira third_party_integrations that help to mitigate common pitfalls:
# - ensure_pyterrier_is_loaded:
#    loads PyTerrier without internet connection
#     (in TIRA, retrieval approaches have no access to the internet to improve reproducibility)
#
# - get_input_directory_and_output_directory:
#   Your software is expected to read the data from an input directory and write the results (i.e., the run file) to an output directory.
#   Both input and output directories are passes as arguments when the software is executed within TIRA,
#   so this command ensures that you can run the same notebook for development as in TIRA by
#   returning the passed input directory (that might be mounted) if the software is not executed in TIRA.
#
# - persist_and_normalize_run:
#   Writing run files can come with some non-obvious edge cases (e.g., score ties).
#   This method takes care of some frequent of those edge cases.
#
# You do not have to use any of those methods, in the end it is only "generate an output from an input".
# We are of course also happy for pull requests that help to improve the handling of frequently used patterns.
# Please find the documentation here: https://github.com/tira-io/tira/blob/main/python-client/tira/third_party_integrations.py
#
from tira.third_party_integrations import ensure_pyterrier_is_loaded, get_input_directory_and_output_directory, persist_and_normalize_run
import json
from tqdm import tqdm

ensure_pyterrier_is_loaded()
input_directory, output_directory = get_input_directory_and_output_directory('./iranthology-dataset-tira')


I will use a small hardcoded example located in ./iranthology-dataset-tira.
The output directory is /tmp/


In [9]:
print('The input directory contains the following files:\n')
!ls -lh {input_directory}

The input directory contains the following files:

total 67M
-rw-r--r-- 1 root root  67M May 30 08:12 anthology_documents.jsonl
-rwxrwxrwx 1 root root 2.3K May  2 08:19 anthology_topics.xml
-rw-r--r-- 1 root root 8.8K May 30 07:51 qrels.txt


### Step 2: Load the Data

In [10]:
print('Step 2: Load the data.')

queries = pt.io.read_topics(input_directory + '/anthology_topics.xml', format='trecxml')

documents = [json.loads(i) for i in open(input_directory + '/anthology_documents.jsonl', 'r')]


Step 2: Load the data.


In [11]:
print('We look at the first document:\n')
print(documents[0])

We look at the first document:

{'docno': '2019.sigirconf_workshop-2019birndl.0', 'text': "'series': 'CEUR Workshop Proceedings', 'volume': '2414', 'publisher': 'CEUR-WS.org', 'year': '2019', 'url': 'http://ceur-ws.org/Vol-2414', 'urn': 'urn:nbn:de:0074-2414-3', 'biburl': 'https://dblp.org/rec/conf/sigir/2019birndl.bib', 'bibsource': 'dblp computer science bibliography, https://dblp.org', 'bibkey': 'DBLP:conf/sigir/2019birndl', 'bibtype': 'proceedings', 'booktitle': 'Proceedings of the 4th Joint Workshop on Bibliometric-enhanced Information Retrieval and Natural Language Processing for Digital Libraries (BIRNDL 2019) co-located with the 42nd International ACM SIGIR Conference on Research and Development in Information Retrieval (SIGIR 2019), Paris, France, July 25, 2019', 'authors': [], 'editors': ['Muthu Kumar Chandrasekaran', 'Philipp Mayr'], 'venue': 'SIGIR', 'date': 1581522299.0, 'abstract': '', 'title': 'Proceedings of the 4th Joint Workshop on Bibliometric-enhanced Information Re

In [12]:
print('We look at the first query:\n')
print(queries.iloc[0].to_dict())

We look at the first query:

{'qid': '1', 'query': ' relevant documents include the words index and or indexing and exhaustivity in combination documents only containing indexing and or index or only containing exhaustivity are not relevant'}


### Step 3: Create the Index

In [13]:
print('Step 3: Create the Index.')

!rm -Rf ./index
iter_indexer = pt.IterDictIndexer("./index", meta={'docno' : 100})
index_ref = iter_indexer.index(tqdm(documents))

Step 3: Create the Index.



  3%|██▏                                                                           | 1479/53673 [00:46<17:32, 49.60it/s]


  9%|██████▋                                                                      | 4625/53673 [01:21<02:26, 333.71it/s]


 16%|████████████▏                                                                | 8481/53673 [01:42<04:11, 179.47it/s]


 25%|██████████████████▉                                                         | 13389/53673 [01:57<00:47, 848.75it/s]


...

 59%|████████████████████████████████████████████▍                               | 31408/53673 [02:30<00:30, 718.33it/s]


 69%|████████████████████████████████████████████████████▊                       | 37273/53673 [02:39<00:19, 833.43it/s]


 84%|██████████████████████████████████████████████████████████████▋            | 44893/53673 [02:49<00:07, 1154.21it/s]


 97%|█████████████████████████████████████████████████████████████████████████▉  | 52206/53673 [03:12<00:01, 923.71it/s]


100%|████████████████████████████████████████████████████████████████████████████| 53673/53673 [03:14<00:00, 275.65it/s]


### Step 4: Create Retrieval Pipeline

In [14]:
bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25", verbose=True)

### Step 5: Create the run

In [15]:
print('Step 5: Create Run.')

run = bm25(queries)

Step 5: Create Run.



BR(BM25): 100%|████████████████████████████████████████████████████████████████████████████| 6/6 [00:11<00:00,  1.93s/q]


In [16]:
print('We look at the first 10 results of the run:\n')
run.head(10)

We look at the first 10 results of the run:



,qid,docid,docno,rank,score,query
0,1,15484,2017.wsdm_conference-2017.24,0,16.608272,relevant documents include the words index an...
1,1,51891,2008.ipm_journal-ir0anthology0volumeA44A4.8,1,16.378209,relevant documents include the words index an...
2,1,37385,2006.trec_conference-2006.9,2,15.981597,relevant documents include the words index an...
3,1,2193,2009.clef_workshop-2009w.90,3,15.922364,relevant documents include the words index an...
4,1,37794,2001.trec_conference-2001.65,4,15.701605,relevant documents include the words index an...
5,1,52854,2015.tois_journal-ir0anthology0volumeA33A4.1,5,15.258457,relevant documents include the words index an...
6,1,15120,2013.wsdm_conference-2013.9,6,15.105135,relevant documents include the words index an...
7,1,22826,2010.cikm_conference-2010.49,7,15.062287,relevant documents include the words index an...
8,1,6275,2011.sigirconf_conference-2011.58,8,14.880142,relevant documents include the words index an...
9,1,52856,2015.tois_journal-ir0anthology0volumeA33A4.3,9,14.837122,relevant documents include the words index an...


### Step 6: Persist Run

In [20]:
print('Step 6: Persist Run.')

persist_and_normalize_run(run, output_file=output_directory, system_name='BM25', depth=1000)

print('Done :)')

Step 6: Persist Run.
Done :)
